In [36]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
np.random.seed(1111)


In [37]:
df = pd.read_csv('../data/ch11_potato.csv')
sample = np.array(df['무게'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [38]:
# 표본평균
s_mean = np.mean(sample)
s_mean

128.451

1. 통계적 가설검정
- 1-1. 통계적 가설검정의 기본

- 귀무가설 : 감자튀김의 평균은 130
- 대립가설 : 평균은 130보다 작음

In [39]:
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.681

표본 평균이 128.681보다 작으면 귀무가설 기각

In [40]:
# 검정통계량
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932

In [41]:
# 임곗값
rv = stats.norm()
rv.isf(0.95)

-1.645

검정통계량이 임곗값 보다 작음 ->  귀무가설 기각, 평균은 130보다 작다고 판단

In [42]:
# p값
rv.cdf(z)

0.027

p값 또한 유의수준 0.05보다 작아 귀무가설은 기각됨

- 1-2. 단측검정과 양측검정

In [43]:
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932

In [44]:
# 임곗값
rv = stats.norm()
rv.interval(0.95)

(-1.960, 1.960)

검정통계량이 채택역이 들어 있음 -> 귀무가설을 기각하지 않음

In [45]:
# 누적밀도함수의 값을 2배로 설정
rv.cdf(z) * 2

0.053

p-value값이 0.05보다 크기 때문에 귀무가설 채택

- 1-3. 가설검정의 두 가지 오류

- 1. 제 1종 오류 : 귀무가설이 옳을 때, 귀무가설을 기각하는 오류
- 2. 제 2종 오류 : 대립가설이 옳을 때, 귀무가설을 채택하는 오류

In [46]:
rv = stats.norm(130,3)

# 14개의 표본을 추출하여 가설검정을 수행하는 작업을 10000번 진행
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0

for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14),2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    
    if z < c:
        cnt += 1

cnt/ n_samples

0.053

제 1종 오류를 범한 비율은 0.053으로 대략 5%의 비율로 130g보다 작다라고 잘못 탐지함

In [47]:
rv = stats.norm(128,3)

# 제 2종 오류를 범하는 비율 계산
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0

for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14),2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    
    if z >= c:
        cnt += 1

cnt / n_samples

0.197

제 2종 오류를 범하는 비율은 0.197로 대략 20%의 비율로 미탐이 발생함

2. 기본적인 가설검정
- 2-1. 정규분포의 모평균에 대한 검정 : 모분산을 알고 있는  경우

In [48]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('귀무가설을 채택')
        
    else: 
        print('귀무가설을 기각')
        
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1-rv.cdf(z)) * 2
    
    print(f'p값은 {p:.3f}')

In [49]:
pmean_test(sample, 130, 9)

귀무가설을 채택
p값은 0.053


- 2-2. 정규분포의 모분산에 대한 검정

In [50]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
        
    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
        
    else:
        p = (1 - rv.cdf(y)) * 2
    
    print(f'p값은 {p:.3f}')

In [51]:
pvar_test(sample, 9)

귀무가설을 채택
p값은 0.085


- 2-3. 정규분포의 모평균에 대한 검정 : 모분산을 모르는 경우

In [52]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)
    
    t = (s_mean - mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
        
    else:
        print('귀무가설을 기각')
        
    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1-rv.cdf(t)) * 2
        
    print(f'p값은 {p:.3f}')

In [53]:
pmean_test(sample, 130)

귀무가설을 채택
p값은 0.169


In [54]:
# 함수 사용
stats.ttest_1samp(sample,130)

Ttest_1sampResult(statistic=-1.4551960206404198, pvalue=0.16933464230414275)

3. 2표본 문제에 관한 가설검정
- 3-1. 대응비교 t검정

In [55]:
training_rel = pd.read_csv('../data/ch11_training_rel.csv')
print(training_rel.shape)
training_rel.head()

(20, 2)


,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


- 귀무가설 : 후 - 전 = 0
- 대립가설 : 0이 아님

In [56]:
training_rel['차'] = training_rel['후'] - training_rel['전']
training_rel.head()

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


In [57]:
stats.ttest_1samp(training_rel['차'], 0)

Ttest_1sampResult(statistic=2.204154108716127, pvalue=0.04004419061842953)

p-value값이 유의수준보다 작기 때문에 귀무가설 기각 -> 근력운동은 집중력의 유의한 차이를 가져옴

ttest_rel() 함수를 사용하면 diff를 따로 구하지 않아도 됨

In [58]:
stats.ttest_rel(training_rel['후'], training_rel['전'])

Ttest_relResult(statistic=2.204154108716127, pvalue=0.04004419061842953)

- 3-2. 독립비교 t검정

In [59]:
training_ind = pd.read_csv('../data/ch11_training_ind.csv')
print(training_ind.shape)
training_ind.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [60]:
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'], equal_var=False)
p

0.087

p-value가 유의수준보다 크기 때문에 귀무가설 채택

- 3-3. 윌콕슨의 부호순위 검정 : 대응표본 차이에서 정규분포를 가정할 수  없는 경우, 중앙값의 ㅊ차이에 대한 검정이다. 대응비교 t검정 때와 달리 중앙값의 차이에 대한 검정이다. 

In [61]:
training_ind = pd.read_csv('../data/ch11_training_ind.csv')
toy_df = training_rel[:6].copy()
toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [62]:
diff = toy_df['후'] - toy_df['전']
toy_df['차'] = diff
toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [63]:
# 절대값이 작은 것부터 순서대로 순위 부여
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [64]:
# 차이의 부호가 마이너스인 것의 순위합과 차이의 부호가 플러스인 것의 순위합을 각각 구함
r_minus = np.sum((diff<0) * rank)
r_plus = np.sum((diff>0) * rank)

r_minus, r_plus

(8, 13)

r_minus와 r_plus 중 작은 것이 검정통계량이 됨 -> 검정통계량은 8

윌콕슨의 부호순위검정에서는 이 검정통계량이 임곗값보다 작은 경우에 귀무가설이 기각되는 단측검정을 수행

In [65]:
# 근력운동 후 모두 집중력이 상승했다고 가정

toy_df['후'] = toy_df['전'] + np.arange(1,7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)

toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [66]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(0, 21)

검정통계량은 0, 차이에 편차가 있으면 검정통계량이 작아짐

In [67]:
# 근력운동 후 테스트 결과가 올라간 사람도 있고 내려간 사람도 있는 상황 가정

toy_df['후'] = toy_df['전'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)

toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [68]:
# 차이가 마이너스인 것의 순위합과 플러스인 것의 순위합 각각

r_minus = np.sum((diff<0) * rank)
r_plus = np.sum((diff>0) * rank)

r_minus, r_plus

(11, 10)

r_minus와 r_plus는 비슷한 값이 됨 

In [69]:
# Wilcoxon 검정

T, p = stats.wilcoxon(training_rel['전'], training_rel['후'])
p

0.036

귀무가설 기각 -> 대응비교 t검정과 동일한 결론

윌콕슨의 부호순위검정은 모집단이 정규분포를 따르는 경우에도 사용가능, 대응비교 t검정에 비해 검정력이 낮음

In [70]:
# N(3,4**2)를 따르고 표본 크기가 20인 표본 데이터 1만개 생성

n = 10000
diffs = np.round(stats.norm(3,4).rvs(size=(n,20)))

In [71]:
cnt = 0
alpha = 0.05

for diff in diffs:
    t, p = stats.ttest_1samp(diff, 0)
    if p < alpha:
        cnt += 1
        
cnt / n

0.883

In [73]:
# wicoxon 

cnt = 0
alpha = 0.05

for diff in diffs:
    T, p = stats.wilcoxon(diff)
    if p < alpha:
        cnt += 1
        
cnt/n

/Users/suhye/micromamba/lib/python3.9/site-packages/scipy/stats/_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


0.873

근소한 차이지만, 대응비교 t검정의 검정력이 큼 -> 모집단이 정규분포를 따르는 경우 대응비교 t검정의 검정력이 좋음

- 3-4. 만 위트니의 U검정 : 대응되는 데이터가 없는 2표본 모집단에 정규분포를 가정할 수 없는 경우, 중앙값의 차이에 대한 검정 (윌콕슨의 순위합검정이라고도 함)

In [74]:
training_ind = pd.read_csv('../data/ch11_training_ind.csv')
toy_df = training_ind[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [75]:
# 데이터 전체에 대해 값이 작은 순서대로 순위를 부여

rank = stats.rankdata(np.concatenate([toy_df['A'],
                                      toy_df['B']]))

rank_df = pd.DataFrame({'A' : rank[:5], 'B':rank[5:10]}).astype(int)
rank_df

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


A에 좋은 순위가 모여있으면 순위합이 작아지고 A에 나쁜 순위가 모여있으면 순위합이 커지기 때문에 순위합은 2표본 사이의 데이터 편향을 잘 반영함

In [76]:
n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1)/2)
u

7.000

In [78]:
# A에 좋은 순위가 모두 모여 있는 경우 

rank_df = pd.DataFrame(np.arange(1,11).reshape(2,5).T,
                       columns=['A','B'])
rank_df

,A,B
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [79]:
u = rank_df['A'].sum() - (n1 * (n1+1))/2
u

0.000

In [81]:
# A에 나쁜 순위가 모두 모여있는 경우
rank_df = pd.DataFrame(np.arange(1,11).reshape(2,5)[::-1].T,
                       columns=['A','B'])
rank_df

,A,B
0,6,1
1,7,2
2,8,3
3,9,4
4,10,5


In [82]:
u = rank_df['A'].sum() - (n1 * (n1+1))/2
u

25.000

A에 좋은 순위만 모여 있는 경우에도, 나쁜 순위가 모여 있는 경우에도 2표본의 중앙값에 편향이 존재함 -> U검정은 양측검정을 수행해야함

In [83]:
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'],
                          alternative='two-sided')
p

0.059

독립비교 t검정과 마찬가지로 귀무가설이 채택됨, U검정은 모집단이 정규분포를 따르는 경우 독립비교 t검정에 비해 검정력이 낮아짐

- 3-5. 카이제곱검정